In [0]:
import asyncio
from azure.servicebus.aio import ServiceBusClient
from azure.servicebus import ServiceBusMessage

In [0]:
NAMESPACE_CONNECTION_STR = dbutils.secrets.get(scope="servicebus", key="connection_string")
QUEUE_NAME = "financial_report_queue"

In [0]:
async def send_batch_message(sender):
    async with sender:
        batch_message = await sender.create_message_batch()
        for _ in range(5):
            try:
                batch_message.add_message(ServiceBusMessage("Message inside a ServiceBusMessageBatch"))
            except ValueError:
                break
        await sender.send_messages(batch_message)
    print("Sent a batch of 5 messages")

In [0]:
async def run_and_send():
    async with ServiceBusClient.from_connection_string(
        conn_str=NAMESPACE_CONNECTION_STR,
        logging_enable=True) as servicebus_client:
        sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
        async with sender:
            await send_batch_message(sender)

In [0]:
await run_and_send()
print("Done sending messages")
print("-----------------------")

In [0]:
async def run_and_receive():
    async with ServiceBusClient.from_connection_string(
        conn_str=NAMESPACE_CONNECTION_STR,
        logging_enable=True) as servicebus_client:

        async with servicebus_client:
            receiver = servicebus_client.get_queue_receiver(queue_name=QUEUE_NAME)
            async with receiver:
                received_msgs = await receiver.receive_messages(max_wait_time=5, max_message_count=450)
                for msg in received_msgs:
                    print("Received: " + str(msg))
                    await receiver.complete_message(msg)

In [0]:
await run_and_receive()
print("Done receiving messages")
print("-----------------------")

In [0]:
active_stocks = spark.sql("SELECT DISTINCT TICKER FROM plstocks.silver_stocks_price WHERE active is TRUE")
tickers_list = [row["TICKER"] for row in active_stocks.collect()]
print(tickers_list)

In [0]:

async def send_active_stocks(connection_string, queue_name):
     async with ServiceBusClient.from_connection_string(
        conn_str=connection_string,
        logging_enable=True) as servicebus_client:
            sender = servicebus_client.get_queue_sender(queue_name=queue_name)
            async with sender:
                batch_message = await sender.create_message_batch()
                for ticker in tickers_list:
                    try:
                        print(f"Sending batch for ticker {ticker}")
                        batch_message.add_message(ServiceBusMessage(ticker))
                    except ValueError:
                        break
                await sender.send_messages(batch_message)
                print("Sent tickers")


In [0]:
await send_active_stocks(connection_string=NAMESPACE_CONNECTION_STR, queue_name=QUEUE_NAME)
print("Done sending tickers")
print("-----------------------")

In [0]:
dividend_connection_str = dbutils.secrets.get(scope="servicebus", key="dividend_connection_string")
dividend_queue_name = "dividend_queue"
await send_active_stocks(connection_string=dividend_connection_str, queue_name=dividend_queue_name)
print("Done sending tickers")
print("-----------------------")

In [0]:
await send_active_stocks(connection_string=NAMESPACE_CONNECTION_STR, queue_name=QUEUE_NAME)
print("Done sending tickers")
print("-----------------------")